### SRT Example

In [80]:
from absbox import Generic

seniorBalance = 1000
srtTrancheBal = 200
srtRate = 0.08
closingDate = "2021-06-15"

periodPrincipal = ("curPoolCollection", None, "Principal","Prepayment","Recovery")

srt01 = Generic(
    "SRT Example"
    ,{"cutoff":"2021-06-01","closing":closingDate,"firstPay":"2021-07-20"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":24
          ,"freq":"Monthly","type":"Level","originDate":"2021-06-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":24
          ,"status":"current"}]],
     }
    ,(("acc01",{"balance":0})
      ,("srtAcc",{"balance":srtTrancheBal})
      ,("dummy",{"balance":0.0})
      ,)
    ,(("A1",{"balance":seniorBalance
             ,"rate":0.00
             ,"originBalance":seniorBalance
             ,"originRate":0.00
             ,"startDate":closingDate
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":srtTrancheBal
             ,"rate":srtRate
             ,"originBalance":srtTrancheBal
             ,"originRate":srtRate
             ,"startDate":closingDate
             ,"rateType":{"Fixed":srtRate}
             ,"bondType":{"Sequential":None}
             }
       ))
    ,tuple()
    ,{"amortizing":[
          # pay prorata to senior and SRT tranch
         ['accrueAndPayInt',"dummy",["A1"],{"support":["facility","originator"]}]
         ,["calcInt","B"]
         ,["payInt","dummy",["B"],{"support":["facility","originator"]}]

         ,["calcBondPrin","dummy",["A1","B"] ,{"limit":{"formula":periodPrincipal},"support":["facility","originator"]}]
         ,["inspect","total pool prin income",periodPrincipal]
         ,["inspect","bond:A1 Bal",("bondDuePrin","A1")]
         ,["inspect","bond:B Bal",("bondDuePrin","B")]
         ,["payPrinWithDue","acc01",["A1"]]         
         ,["payPrinWithDue","srtAcc",["B"]]
         #,["inspect","after pay bond:A1 Bal",("bondDuePrin","A1")]
         #,["inspect","after pay bond:B Bal",("bondDuePrin","B")]
     ],
      "endOfCollection":[
          ["liqRepayResidual","srtAcc", "originator", {"formula":("curPoolCollection",None,"Losses")}]
          #["transfer","srtAcc","",{"formula":("cumPoolCollection","Losses")}]
      ],
      "cleanUp":[
          ["payIntResidual","srtAcc","B"]
      ]
     }
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,{"originator":{"type" : "Unlimited", "start": closingDate}}
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

In [90]:
from absbox import API,EnginePath

localAPI = API(EnginePath.LOCAL, lang='english', check=False)

r = localAPI.run(srt01
                ,poolAssump=("Pool",("Mortgage", {"CDR":0.01}, None, None, None)
                                   ,None
                                   ,None)
                ,runAssump=[("inspect"#,(["DayOfMonth",20],pctOfSrt)
                                      ,(["DayOfMonth",20],("poolFactor",)))
                            # ,("stop","2023-01-01")
                            #,("call",("poolFactor",0.20))
                           ]
                ,read=True)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.26.4, server:0.27.11

Warning Message from server:
Bond B is not paid off
Account acc01 has cash to be distributed
LiquidityProvider originator is not paid off

In [57]:
from absbox import readBondsCf

In [82]:
yy = readBondsCf(r['bonds'])
yy

Bond             A1                                      B                     \
Field       balance interest principal rate   cash balance interest principal   
date                                                                            
2021-07-20  1000.00      0.0      0.00  0.0   0.00  200.00     1.53      0.00   
2021-08-20   929.37      0.0     70.63  0.0  70.63  185.88     1.35     14.12   
2021-09-20   858.33      0.0     71.04  0.0  71.04  171.67     1.26     14.21   
2021-10-20   786.88      0.0     71.45  0.0  71.45  157.38     1.12     14.29   
2021-11-20   715.01      0.0     71.87  0.0  71.87  143.01     1.06     14.37   
2021-12-20   642.72      0.0     72.29  0.0  72.29  128.55     0.94     14.46   
2022-01-20   570.01      0.0     72.71  0.0  72.71  114.01     0.87     14.54   
2022-02-20   496.87      0.0     73.14  0.0  73.14   99.39     0.77     14.62   
2022-03-20   423.31      0.0     73.56  0.0  73.56   84.68     0.60     14.71   
2022-04-20   349.32      0.0     73.99  0.0  73.99   69.88     0.57     14.80   
2022-05-20   274.89      0.0     74.43  0.0  74.43   55.00     0.45     14.88   
2022-06-20   200.03      0.0     74.86  0.0  74.86   40.03     0.37     14.97   
2022-07-20   124.75      0.0     75.28  0.0  75.28   24.97     0.26     15.06   
2022-08-20    49.03      0.0     75.72  0.0  75.72   18.11     0.16      6.86   
2022-09-20     0.00      0.0     49.03  0.0  49.03   18.11     0.12      0.00   
2022-10-20      NaN      NaN       NaN  NaN    NaN   18.11     0.11      0.00   
2022-11-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2022-12-20      NaN      NaN       NaN  NaN    NaN   18.11     0.11      0.00   
2023-01-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2023-02-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2023-03-20      NaN      NaN       NaN  NaN    NaN   18.11     0.11      0.00   
2023-04-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2023-05-20      NaN      NaN       NaN  NaN    NaN   18.11     0.11      0.00   
2023-06-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2023-07-20      NaN      NaN       NaN  NaN    NaN   18.11     0.11      0.00   
2023-08-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2023-09-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2023-10-20      NaN      NaN       NaN  NaN    NaN   18.11     0.11      0.00   
2023-11-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2023-12-20      NaN      NaN       NaN  NaN    NaN   18.11     0.11      0.00   
2024-01-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2024-02-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2024-03-20      NaN      NaN       NaN  NaN    NaN   18.11     0.11      0.00   
2024-04-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2024-05-20      NaN      NaN       NaN  NaN    NaN   18.11     0.11      0.00   
2024-06-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2024-07-20      NaN      NaN       NaN  NaN    NaN   18.11     0.11      0.00   
2024-08-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2024-09-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2024-10-20      NaN      NaN       NaN  NaN    NaN   18.11     0.11      0.00   
2024-11-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2024-12-20      NaN      NaN       NaN  NaN    NaN   18.11     0.11      0.00   
2025-01-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2025-02-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2025-03-20      NaN      NaN       NaN  NaN    NaN   18.11     0.11      0.00   
2025-04-20      NaN      NaN       NaN  NaN    NaN   18.11     0.12      0.00   
2025-05-20      NaN      NaN   

In [59]:
xxx = yy[[("A1","balance"),("A1","principal"),("B","balance"),("B","principal")]]
xxx[("Sum Bal","Sum Bal")] = xxx[("A1","balance")] + xxx[("B","balance")] 
xxx[("Sum Bal","Sum Prin")] = xxx[("A1","principal")] + xxx[("B","principal")] 

xxx[("Sum Bal","A1 Factor")] = xxx[("A1","balance")] / xxx[("Sum Bal","Sum Bal")] 
xxx[("Sum Bal","B Factor")] = xxx[("B","balance")] / xxx[("Sum Bal","Sum Bal")] 

xxx[("Sum Bal","A1 Factor")] = xxx[("A1","principal")] / xxx[("Sum Bal","Sum Prin")] 
xxx[("Sum Bal","B Factor")] = xxx[("B","principal")] / xxx[("Sum Bal","Sum Prin")] 

/var/folders/pb/0lsr_g9s0x340sd74jhjdhj40000gn/T/ipykernel_5938/920834521.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xxx[("Sum Bal","Sum Bal")] = xxx[("A1","balance")] + xxx[("B","balance")]
/var/folders/pb/0lsr_g9s0x340sd74jhjdhj40000gn/T/ipykernel_5938/920834521.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xxx[("Sum Bal","Sum Prin")] = xxx[("A1","principal")] + xxx[("B","principal")]
/var/folders/pb/0lsr_g9s0x340sd74jhjdhj40000gn/T/ipykernel_5938/920834521.py:5: SettingWithCopyWarning: 
A 

In [60]:
xxx

Bond             A1                 B            Sum Bal                     \
Field       balance principal balance principal  Sum Bal Sum Prin A1 Factor   
date                                                                          
2021-07-20  1000.00      0.00  200.00      0.00  1200.00     0.00       NaN   
2021-08-20   929.37     70.63  185.88     14.12  1115.25    84.75  0.833392   
2021-09-20   858.33     71.04  171.67     14.21  1030.00    85.25  0.833314   
2021-10-20   786.88     71.45  157.38     14.29   944.26    85.74  0.833333   
2021-11-20   715.01     71.87  143.01     14.37   858.02    86.24  0.833372   
2021-12-20   642.72     72.29  128.55     14.46   771.27    86.75  0.833314   
2022-01-20   570.01     72.71  114.01     14.54   684.02    87.25  0.833352   
2022-02-20   496.87     73.14   99.39     14.62   596.26    87.76  0.833409   
2022-03-20   423.31     73.56   84.68     14.71   507.99    88.27  0.833352   
2022-04-20   349.32     73.99   69.88     14.80   419.20    88.79  0.833315   
2022-05-20   274.89     74.43   55.00     14.88   329.89    89.31  0.833389   
2022-06-20   200.03     74.86   40.03     14.97   240.06    89.83  0.833352   
2022-07-20   124.75     75.28   24.97     15.06   149.72    90.34  0.833296   
2022-08-20    49.03     75.72   17.83      7.14    66.86    82.86  0.913831   
2022-09-20     0.00     49.03   17.83      0.00    17.83    49.03  1.000000   
2022-10-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2022-11-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2022-12-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2023-01-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2023-02-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2023-03-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2023-04-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2023-05-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2023-06-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2023-07-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2023-08-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2023-09-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2023-10-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2023-11-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2023-12-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2024-01-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2024-02-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2024-03-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2024-04-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2024-05-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2024-06-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2024-07-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2024-08-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2024-09-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2024-10-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2024-11-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2024-12-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2025-01-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2025-02-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2025-03-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2025-04-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2025-05-20      NaN       NaN   17.83      0.00      NaN      NaN       NaN   
2025-06-20      NaN       NaN   17.83      0.00   

In [61]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-06-15,2200.00,0.00,0.00,0,0.00,0,0.00,0.08,None,None,0.00,0,0,0.00,0,0.00
2021-06-30,2200.00,0.00,0.00,0,0.00,0,0.00,0.08,None,None,0.00,0,0,0.00,0,0.00
2021-07-31,2113.43,84.76,14.65,0,1.81,0,1.81,0.08,None,None,84.76,0,0,1.81,0,1.81
2021-08-31,2026.37,85.26,14.07,0,1.80,0,1.80,0.08,None,None,170.02,0,0,3.61,0,3.61
2021-09-30,1938.90,85.75,13.49,0,1.72,0,1.72,0.08,None,None,255.77,0,0,5.33,0,5.33
2021-10-31,1851.05,86.25,12.91,0,1.60,0,1.60,0.08,None,None,342.02,0,0,6.93,0,6.93
2021-11-30,1762.72,86.76,12.32,0,1.57,0,1.57,0.08,None,None,428.78,0,0,8.50,0,8.50
2021-12-31,1674.01,87.26,11.74,0,1.45,0,1.45,0.08,None,None,516.04,0,0,9.95,0,9.95
2022-01-31,1584.82,87.77,11.14,0,1.42,0,1.42,0.08,None,None,603.81,0,0,11.37,0,11.37


In [62]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [88]:
r['accounts']['srtAcc'].loc[lambda df: df.memo == "<Support:originator>"]

,balance,change,memo
date,,,
2021-06-15,200.00,0.00,<Support:originator>
2021-06-30,200.00,0.00,<Support:originator>
2021-07-31,198.19,-1.81,<Support:originator>
2021-08-31,182.27,-1.80,<Support:originator>
2021-09-30,166.34,-1.72,<Support:originator>
2021-10-31,150.45,-1.60,<Support:originator>
2021-11-30,134.51,-1.57,<Support:originator>
2021-12-31,118.60,-1.45,<Support:originator>
2022-01-31,102.64,-1.42,<Support:originator>


In [84]:
r['accounts']['srtAcc'].loc[lambda df: df.memo == "<Support:originator>"].change.sum()

-18.11

In [87]:
r['pool']['flow'].Loss

Date
2021-06-15    0.00
2021-06-30    0.00
2021-07-31    1.81
2021-08-31    1.80
2021-09-30    1.72
2021-10-31    1.60
2021-11-30    1.57
2021-12-31    1.45
2022-01-31    1.42
2022-02-28    1.35
2022-03-31    1.15
2022-04-30    1.19
2022-05-31    1.08
2022-06-30    1.04
2022-07-31    0.93
2022-08-31    0.88
2022-09-30    0.80
2022-10-31    0.70
2022-11-30    0.65
2022-12-31    0.55
2023-01-31    0.49
2023-02-28    0.40
2023-03-31    0.29
2023-04-30    0.24
2023-05-31    0.15
2023-06-30    0.08
Name: Loss, dtype: float64

In [6]:
from absbox.local.util import unifyTs

In [7]:
unifyTs(r['result']['inspect'].values())

,<PoolFactor>
Date,
2021-06-01,0.000000
2021-06-20,1.000000
2021-07-20,1.000000
2021-08-20,0.960650
2021-09-20,0.921077
2021-10-20,0.881318
2021-11-20,0.841386
2021-12-20,0.801236
2022-01-20,0.760913


In [33]:
r['result']['waterfallInspect']

,Date,Comment,DealStats,Value
0,2021-07-20,total pool prin income,"{'tag': 'PoolCurCollection', 'contents': [['Co...",0.00
1,2021-07-20,bond:A1 Bal,"{'tag': 'BondDuePrin', 'contents': ['A1']}",0.00
2,2021-07-20,bond:B Bal,"{'tag': 'BondDuePrin', 'contents': ['B']}",0.00
3,2021-07-20,after pay bond:A1 Bal,"{'tag': 'BondDuePrin', 'contents': ['A1']}",0.00
4,2021-07-20,after pay bond:B Bal,"{'tag': 'BondDuePrin', 'contents': ['B']}",0.00
5,2021-08-20,total pool prin income,"{'tag': 'PoolCurCollection', 'contents': [['Co...",84.76
6,2021-08-20,bond:A1 Bal,"{'tag': 'BondDuePrin', 'contents': ['A1']}",70.63
7,2021-08-20,bond:B Bal,"{'tag': 'BondDuePrin', 'contents': ['B']}",14.12
8,2021-08-20,after pay bond:A1 Bal,"{'tag': 'BondDuePrin', 'contents': ['A1']}",0.00
9,2021-08-20,after pay bond:B Bal,"{'tag': 'BondDuePrin', 'contents': ['B']}",0.00
